In [1]:
import pandas as pd
import seaborn as sns
import warnings
import pickle
from matplotlib import pyplot as plt
from IPython.core.display import HTML
from scipy import stats as ss

warnings.filterwarnings( 'ignore' )

In [2]:
%matplotlib inline
%pylab inline

plt.style.use( 'bmh' )
plt.rcParams['figure.figsize'] = [25, 12]
plt.rcParams['font.size'] = 10

Populating the interactive namespace from numpy and matplotlib


In [5]:
with open('code6_cols_selected_boruta.pkl', 'rb') as f:
    cols_selected_boruta = pickle.load(f)
with open('code6_feat_to_add.pkl', 'rb') as f:
    feat_to_add = pickle.load(f)

df7 = pd.read_pickle('code6_df6.pkl')
X_train = pd.read_pickle('code6_X_train.pkl')
X_test = pd.read_pickle('code6_X_test.pkl')
Y_train = pd.read_pickle('code6_Y_train.pkl')
Y_test = pd.read_pickle('code6_Y_test.pkl')

# 7.0 ML Analysis and Techniques

In [24]:
cols_selected_boruta=['store',
 'promo',
 'store_type',
 'assortment',
 'competition_distance',
 'competition_open_since_month',
 'competition_open_since_year',
 'promo2',
 'promo2_since_week',
 'promo2_since_year',
 'competition_time_month',
 'promo_time_week',
 'day_of_week_sin',
 'day_of_week_cos',
 'month_cos',
 'month_sin',
 'day_sin',
 'day_cos',
 'week_of_year_sin',
 'week_of_year_cos']

In [27]:
x_train=X_train[cols_selected_boruta]
x_test=X_test[cols_selected_boruta]

## 7.1 Average Model

In [36]:
def mean_absolute_percentage_error(y,yhat):
    return np.mean(np.abs((y-yhat)/y))

def ml_error(model_name,y,yhat):
    from sklearn.metrics import mean_absolute_error,mean_squared_error
    
    mae=mean_absolute_error(y,yhat)
    mape=mean_absolute_percentage_error(y,yhat)
    rmse=np.sqrt(mean_squared_error(y,yhat))
    
    return pd.DataFrame({'Model name': model_name,
                        'MAE': mae,
                        'MAPE': mape,
                        'RMSE': rmse}, index=[0])

In [38]:
aux1=x_test.copy()
aux1['sales']=Y_test.copy()

#prediciton
aux2=aux1[['store','sales']].groupby('store').mean().reset_index().rename(columns={'sales':'prediction'})
aux1=pd.merge(aux1,aux2,how='left',on='store')
#the preditions 'y_hat'
yhat_baseline=aux1['prediction']

#performance
# nas transformações a classe foi normalizada pela logaritmica, agora para avaliar aperformance, 
# precisamos resgatar o valor original das vendas, aplicando a transformação inversa, no caso a exponencial
baseline_result = ml_error('Average Model',np.expm1(Y_test),np.expm1(yhat_baseline))
baseline_result

,Model name,MAE,MAPE,RMSE
0,Average Model,1354.800353,0.455051,1835.135542


## 7.2 Linear Regression

In [41]:
from sklearn.linear_model import LinearRegression

#model
lr=LinearRegression().fit(x_train,Y_train)

#prediction
yhat_lr=lr.predict(x_test)

#performance
lr_result=ml_error('Linear Regression',np.expm1(Y_test),np.expm1(yhat_lr))
lr_result

,Model name,MAE,MAPE,RMSE
0,Linear Regression,1867.654504,0.292836,2671.333847


## 7.3. Linear Regression Regularized

In [43]:
from sklearn.linear_model import Lasso

#model
# the higher the alpha, the most feature coefficients are zero, that is more generalised
lrr=Lasso(alpha=0.001).fit(x_train,Y_train)

#prediction
yhat_lrr=lrr.predict(x_test)

#performance
lrr_result=ml_error('Linear Regression Lasso',np.expm1(Y_test),np.expm1(yhat_lrr))
lrr_result

,Model name,MAE,MAPE,RMSE
0,Linear Regression Lasso,1870.005016,0.28825,2694.092897


Performance da regressão linear foi de maior erro que a média, o que indica que esse comportamento provavelmente não é linear.

## 7.4. Random Forest

In [17]:
# df6.to_pickle('df6.pkl')
# num_attributes.to_pickle('num_attributes2.pkl')
# cat_attributes.to_pickle('cat_attributes2.pkl')